# pandas 高级应用

## 分类数据

### 背景和目的

表中的一列通常会有重复的包含不同值的小集合的情况

- unique和value_counts，它们可以从数组提取出不同的值，并分别计算频率

在数据仓库中，表征重复值最好的方法是使用所谓的包含不同值的维表(Dimension Table)，将主要的参数存储为引用维表整数键

In [6]:
import pandas as pd
import numpy as np

In [2]:
values = pd.Series([0, 1, 0, 0] * 2)

In [3]:
dim = pd.Series(['apple', 'orange'])

In [4]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

这种用整数表示的方法称为分类或字典编码表示法。
- 不同值的数组称为分类、字典或数据级。
- 表示分类的整数值称为分类编码或简单地称为编码。

### pandas的分类类型

pandas有一个特殊的分类类型category，用于保存使用整数分类表示法的数据
- 分类对象有categories和codes属性
- 分类数组可以包括任意不可变类型

In [7]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2

In [8]:
N = len(fruits)

In [12]:
df = pd.DataFrame({'fruit':fruits,
                  'basket_id':np.arange(N),
                  'count':np.random.randint(3, 15, size=N),
                  'wieght':np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])

In [13]:
df

,basket_id,fruit,count,weight
0,0,apple,3,NaN
1,1,orange,9,NaN
2,2,apple,6,NaN
3,3,apple,11,NaN
4,4,apple,10,NaN
5,5,orange,13,NaN
6,6,apple,4,NaN
7,7,apple,7,NaN


In [14]:
fruit_cat = df['fruit']

In [15]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: object

In [17]:
fruit_cat = fruit_cat.astype('category')

In [19]:
c = fruit_cat.values

In [20]:
c

[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]

In [21]:
type(c)

pandas.core.arrays.categorical.Categorical

In [22]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [23]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

- 可将DataFrame的列通过分配转换结果，转换为分类
- 可以从其它Python序列直接创建pandas.Categorical

In [24]:
df.fruit = df.fruit.astype('category')

In [25]:
df

,basket_id,fruit,count,weight
0,0,apple,3,NaN
1,1,orange,9,NaN
2,2,apple,6,NaN
3,3,apple,11,NaN
4,4,apple,10,NaN
5,5,orange,13,NaN
6,6,apple,4,NaN
7,7,apple,7,NaN


In [26]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])

In [27]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

可以使用from_codes构造器构造分类编码

与显示指定不同，分类变换不认定指定的分类顺序。当使用from_codes或其它的构造器时，你可以指定分类一个有意义的顺序

In [28]:
categories = ['foo', 'bar', 'baz']

In [31]:
codes = [0, 1, 2, 0, 0, 1]

In [32]:
my_cat_2 = pd.Categorical.from_codes(codes, categories)

In [33]:
my_cat_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

In [34]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

In [35]:
ordered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

### 用分类进行计算

某些pandas组件，比如groupby函数，更适合进行分类。

In [36]:
np.random.seed(12345)

In [37]:
draws = np.random.randn(1000)

In [38]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [39]:
bins = pd.qcut(draws, 4)

In [40]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

确切的样本分位数与分位的名称相比，不利于生成汇总。我们可以使用labels参数qcut

In [41]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [42]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [44]:
bins = pd.Series(bins, name='quartile')

In [46]:
results = pd.Series(draws).groupby(bins).agg(['min', 'max', 'mean'])

In [48]:
results.reset_index()

,quartile,min,max,mean
0,Q1,-2.949343,-0.685484,-1.215981
1,Q2,-0.683066,-0.010115,-0.362423
2,Q3,-0.010032,0.628894,0.307840
3,Q4,0.634238,3.927528,1.261160


### 用分类提高性能

如果你是在一个特定数据集上做大量分析，将其转换为分类可以极大地提高效率。

DataFrame列的分类使用的内存通常少的多。

In [49]:
N = 10000000

In [50]:
draws = pd.Series(np.random.randn(N))

In [52]:
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [53]:
categories = labels.astype('category')

In [54]:
labels.memory_usage()

80000080

In [55]:
categories.memory_usage()

10000272

In [56]:
time categories = labels.astype('category')

Wall time: 842 ms


>GroupBy使用分类操作明显更快，是因为底层的算法使用整数编码数组，而不是字符串数组。

### 分类方法

特别的cat属性提供了分类方法的入口

![](cat方法.jpg)

### 为建模创建虚拟变量

使用统计或机器学习工具时，通常会将分类数据转换为虚拟变量，也称为one-hot编码。

pandas.get_dummies函数可以转换这个分类数据为包含虚拟变量的DataFrame

In [57]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

In [58]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## GroupBy高级应用

### 分组转换和“解封”GroupBy

transform方法，它与apply很像，但是对使用的函数有一定限制

- 它可以产生向分组形状广播标量值
- 它可以产生一个和输入组形状相同的对象
- 它不能修改输入

内置的聚合函数，比如mean或sum，通常比apply函数快，也比transform快。
- 我们可以通过多个子操作来达到同样的效果，矢量化可以减少时间

In [60]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
....: 'value': np.arange(12.)})

In [61]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [62]:
g = df.groupby('key').value

In [63]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [70]:
timeit g.transform(normalize)

4.68 ms ± 539 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [69]:
timeit (df['value'] - g.transform('mean')) / g.transform('std')

4.23 ms ± 201 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 分组的时间重采样

In [71]:
pass

## 链式编程技术

当对数据集进行一系列变换时，你可能发现创建的多个临时变量其实并没有在分析中用到。

DataFrame.assign方法不是就地修改对象，而是返回新的修改过的DataFrame

下面的语句是等价的
```python
# Usual non-functional way
df2 = df.copy()
df2['k'] = v
# Functional assign way
df2 = df.assign(k=v)
```

assign和许多其它pandas函数可以接收类似函数的参数，即可调用对象（callable）。

```python
df = load_data()
df2 = df[df['col2'] < 0]
df2['col1_demeaned'] = df2['col1'] - df2['col1'].mean()
result = df2.groupby('key').col1_demeaned.std()
```

```python
# 也可以写成
result = (load_data()
        [lambda x: x.col2 < 0]
        .assign(col1_demeaned=lambda x: x.col1 - x.col1.mean())
        .groupby('key')
        .col1_demeaned.std())
```

>是否将代码写成这种形式只是习惯而已，将它分开成若干步可以提高**可读性**。

### 管道方法

可以用Python内置的pandas函数和方法，用带有可调用对象的链式编程。

但是，有时你需要使用自己的函数，或是第三方库的函数。这时就要用到管道方法。

下面两端代码是等价的：
```
a = f(df, arg1=v1)
b = g(a, v2, arg3=v3)
c = h(b, arg4=v4)
```

```
result = (df.pipe(f, arg1=v1)
.pipe(g, v2, arg3=v3)
.pipe(h, arg4=v4))
```

pipe的另一个有用的地方是提炼操作为可复用的函数

```
g = df.groupby(['key1', 'key2'])
df['col1'] = df['col1'] - g.transform('mean')
```
将其抽象化为函数
```
def group_demean(df, by, cols):
    result = df.copy()
    g = df.groupby(by)
    for c in cols:
        result[c] = df[c] - g[c].transform('mean')
    return result
```
可以写成：
`result = (df[df.col1 < 0].pipe(group_demean, ['key1', 'key2'], ['col1']))`

看了科赛的作业，感觉比较j